In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import re
import sys
import wiki_article 
from dataclasses import dataclass
from io import StringIO
from transformers import AutoModelWithLMHead, AutoTokenizer
from scipy import stats

In [305]:
interleave = 768
wiki_article.generate_text_dataset(
    open("wikitext-2-raw/wiki.train.raw", "r"), 
    open("wikitext_title_train/wiki_title.train.raw", "w"),
    interleave=interleave,
)
wiki_article.generate_text_dataset(
    open("wikitext-2-raw/wiki.valid.raw", "r"), 
    open("wikitext_title_train/wiki_title.valid.raw", "w"),
    interleave=interleave,
)

In [27]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2").to('cuda')

In [9]:
unrefined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
unrefined_model_eval

DescribeResult(nobs=60, minmax=(9.851899147033691, 15.262736320495605), mean=12.927889172236124, variance=1.572701685821396, skewness=-0.44557272655893687, kurtosis=-0.35688076179019035)

In [30]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("output_103/").to('cuda')

In [17]:
refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
refined_model_eval

DescribeResult(nobs=60, minmax=(10.398584365844727, 17.0587158203125), mean=13.77767310142517, variance=1.9294105073945371, skewness=-0.01741720699601202, kurtosis=-0.0981178051223095)

In [102]:
sequence = f"\"TITLE\" is a song collaboration by Chinese artist Pamela Chen and Canadian singer Thomas Dimson, first released independently in March 2020. After gaining popularity amongst the cat community, the single was re-released by major label Columbia Records in May 2020. Pamela describes the song as being originally inspired by her two kittens, Apollo and Bean who once said meow.<bot>"

input = tokenizer.encode(sequence, return_tensors="pt").to('cuda')
generated = model.generate(input, max_length=100, num_return_sequences=10, temperature=1)

print(f"Prompt text: {sequence}")
for i in range(generated.size()[0]):
    sentence_tokens = generated[i, :].tolist()
    decoded = tokenizer.decode(sentence_tokens)
    m = re.search(r"<bot>(.*?)<eot>", decoded)
    if m:
        print(f"{i}) {m.groups(1)}")
    else:
        print(f"{i}) Didn't work")
    

resulting_string = tokenizer.decode(generated.tolist()[0])
# print(resulting_string)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Prompt text: "TITLE" is a song collaboration by Chinese artist Pamela Chen and Canadian singer Thomas Dimson, first released independently in March 2020. After gaining popularity amongst the cat community, the single was re-released by major label Columbia Records in May 2020. Pamela describes the song as being originally inspired by her two kittens, Apollo and Bean who once said meow.<bot>
0) ('Animal Heart',)
1) ('The Song of a Cuddle',)
2) ('Letting You Come',)
3) ('Escape from Cats',)
4) ("So You Think It's All Good",)
5) ("Don 't Worry About You",)
6) ('I Am... TITLE',)
7) ('I Put a Smile on a Smile Baby',)
8) ('I Wanna Be',)
9) ('Pump the Pump',)
